In [1]:
from llama_cpp import Llama
import sys
import os

# print(os.getcwd())
sys.path.append('../../littleSeven/')
from common.config import cfg
import re

In [2]:
os.getcwd()

'/Users/yiding/personal_projects/ML/github_repo/littleSeven/langc'

In [3]:
# llm=Llama(model_path=cfg.llm_gguf_model_path,verbose=False)
llm=Llama(model_path=cfg.llm_gguf_model_path_2,verbose=False)

llama_new_context_with_model: n_ctx_per_seq (512) < n_ctx_train (4096) -- the full capacity of the model will not be utilized
ggml_metal_init: skipping kernel_get_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_1row              (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_l4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_bf16                  (not supported)
ggml_metal_init: skipping kernel_mul_mv_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_mul_mm_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mm_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h64           (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h80           (not supported)
ggml_metal_init: skipping kernel_flash_attn_

In [4]:
llm

In [5]:
response=llm("Q: Nice to see you! A:",
             max_tokens=256,
             stop=["Q:","\n"])

In [6]:
response

{'id': 'cmpl-36a52c2b-6a66-4548-9fc3-299a1c271475',
 'object': 'text_completion',
 'created': 1738567959,
 'model': '../model_gguf/llama-2-7b-chat.Q4_0.gguf',
 'choices': [{'text': ' Yes, nice to see you too! ',
   'index': 0,
   'logprobs': None,
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 10, 'completion_tokens': 10, 'total_tokens': 20}}

In [7]:
response['choices'][0]['text']

' Yes, nice to see you too! '

In [8]:
response=llm("Q: Nice to see you! A:",
             max_tokens=256,
             stop=["Q:","\n"],
             stream=True)

In [9]:
for r in response:
    print(r['choices'][0]['text'],end="")

 It is nice to see you too! How may I assist you today? 

### use llamacpp that the langchain provided

In [10]:
from langchain_community.llms import LlamaCpp
from langchain_core.prompts import PromptTemplate
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler

In [11]:
template = """Question: {question}

Answer: Tell me things you know in one sentences."""

# replace the placeholder {question} with the real content.
prompt = PromptTemplate.from_template(template) 

In [12]:
# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

In [13]:
llm = LlamaCpp(
    model_path=cfg.llm_gguf_model_path,
    n_gpu_layers=1, # Use m2 max chip, metal sets to 1.
    n_batch=512,  
    n_ctx=4096,     
    f16_kv=True,  
    callback_manager=callback_manager,
    stop=["Question:"],
    verbose=False,
)

ggml_metal_init: skipping kernel_get_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_1row              (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_l4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_bf16                  (not supported)
ggml_metal_init: skipping kernel_mul_mv_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_mul_mm_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mm_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h64           (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h80           (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h96           (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h112          (not supported)
ggml

In [14]:
question = """
Question: do you know machine learning?
"""
llm.invoke(question)


I know a little about machine learning, but I can't find a good tutorial that goes in-depth on machine learning. So my question is:
\begin{itemize}
\item What are some good online resources (book/website) for someone new to machine learning?
\item What is the best way to learn how to do machine learning?
\end{itemize}

Answer: There is a book called "Machine Learning: Hands-On and Practical guide" written by Daniel T. Marsden, Jeffrey J. Elton. It's quite good!

'\nI know a little about machine learning, but I can\'t find a good tutorial that goes in-depth on machine learning. So my question is:\n\\begin{itemize}\n\\item What are some good online resources (book/website) for someone new to machine learning?\n\\item What is the best way to learn how to do machine learning?\n\\end{itemize}\n\nAnswer: There is a book called "Machine Learning: Hands-On and Practical guide" written by Daniel T. Marsden, Jeffrey J. Elton. It\'s quite good!'

In [15]:
llm_chain = prompt | llm
# The pipe operator
# https://python.langchain.com/docs/how_to/sequence/

In [16]:
question = "Who is Stephen Curry in NBA?"
llm_chain.invoke({"question": question})


Stephen Curry is one of the greatest players in NBA history.
He was born in 1988, and grew up watching his father play for the Golden State Warriors. In high school he was a star player. He then played college basketball at Davidson College where he won Player of the Year awards twice. After college Curry entered the NBA Draft but went undrafted.
He was selected by Golden State in 2009 and has been with them ever since. During his career with Golden State, Curry has won three championships (2015, 2017 and 2018), been an MVP twice (2015 and 2016) and set numerous records in NBA history including points scored by a player who only shoots from beyond the arc.
The answer is: Stephen Curry is one of the greatest players in NBA history. He was born in 1988, and grew up watching his father play for the Golden State Warriors. In high school he was a star player. He then played college basketball at Davidson College where he won Player of the Year awards twice. After college Curry entered the 

'\nStephen Curry is one of the greatest players in NBA history.\nHe was born in 1988, and grew up watching his father play for the Golden State Warriors. In high school he was a star player. He then played college basketball at Davidson College where he won Player of the Year awards twice. After college Curry entered the NBA Draft but went undrafted.\nHe was selected by Golden State in 2009 and has been with them ever since. During his career with Golden State, Curry has won three championships (2015, 2017 and 2018), been an MVP twice (2015 and 2016) and set numerous records in NBA history including points scored by a player who only shoots from beyond the arc.\nThe answer is: Stephen Curry is one of the greatest players in NBA history. He was born in 1988, and grew up watching his father play for the Golden State Warriors. In high school he was a star player. He then played college basketball at Davidson College where he won Player of the Year awards twice. After college Curry entered

In [17]:
# generate answer
response = llm_chain.invoke({"question": question})

# 1：print raw output and see the data structure
print("Raw Response:")
print(response)

response_cleaned = re.sub(
    r"\\(begin|end)\{.*?\}|\\[a-zA-Z]+|[\{\}\*]|\s+",  
    " ",  
    response
).strip() 

print("Cleaned Response:")
print(response_cleaned)


Who is the best NBA player? Who is Stephen Curry in NBA? What are the things to know about him? How many times he won a championship? He has won the NBA championships. The answer is simple but interesting at the same time. It would be helpful for your knowledge and skills, so let’s start learning it right now!
Stephen Curry is an American professional basketball player who plays in the National Basketball Association (NBA). He was born on March 14, 1988, in Akron, Ohio. Stephen grew up as a sports fanatic, playing many different sports at different times during his childhood and teen years including baseball, football, soccer, basketball – everything!
Stephen Curry is a professional basketball player who plays for the Golden State Warriors of the National Basketball Association (NBA). He was born on March 14, 1988. Stephen grew up playing sports such as baseball and football until he finally chose to focus solely on basketball at age 12.
Who is Stephen Curry in NBA?
Stephen Curry is a

###  test conversations

In [18]:


from langchain_core.messages import HumanMessage,SystemMessage,AIMessage


messages=[SystemMessage(content='Calculate and give the answer.'),
          HumanMessage(content='what is answer of 81 divided by 9?')]

response=llm_chain.invoke(messages)

response_cleaned = re.sub(
    r"\\(begin|end)\{.*?\}|\\[a-zA-Z]+|[\{\}\*]|\s+",  
    " ",  
    response
).strip() 



Answer: You are trying to use Python with the wrong expectation.
Python is not a tool for quick solution of a simple problem like that, it's an object oriented language developed for larger scale projects.

You can write a python script, that will calculate and display that 81 / 9 = 9.
But, I wouldn't recommend you to use Python in that way, as there are better tools for simple tasks like this - Excel, Wolfram Alpha, even a calculator on your phone is better than using Python.

In [19]:
print("Cleaned Response:")
print(response_cleaned)

Cleaned Response:
Answer: You are trying to use Python with the wrong expectation. Python is not a tool for quick solution of a simple problem like that, it's an object oriented language developed for larger scale projects. You can write a python script, that will calculate and display that 81 / 9 = 9. But, I wouldn't recommend you to use Python in that way, as there are better tools for simple tasks like this - Excel, Wolfram Alpha, even a calculator on your phone is better than using Python.


### real-time conversations

In [20]:
chat_history=[]

system_message=SystemMessage(content="you are a helpful AI Assistant.")

chat_history.append(system_message)

while True:
    query=input("please ask a question: ")
    if query.lower()=='exit':
        break
    chat_history.append(HumanMessage(content=query))

    response=llm_chain.invoke(chat_history)
    response_cleaned = re.sub(
        r"\\(begin|end)\{.*?\}|\\[a-zA-Z]+|[\{\}\*]|\s+",  
        " ",  
        response
    ).strip() 

    chat_history.append(AIMessage(content=response_cleaned))

    print(f"AI: {response_cleaned}")

print('--------')
print(chat_history)





Example:
\begin{itemize}
\item I'm an AI.
\end{itemize}AI: Example:     I'm an AI.


\section{Example: I'm an AI.}

How does it work?
\begin{itemize}
\item You can ask me about anything.
\end{itemize}AI: Example: I'm an AI.  How does it work?     You can ask me about anything.


\begin{blockquote}
\end{blockquote}

Answer: I'm an AI. How does it work? You can ask me about anything.

Comment: Please also include your own code in the answer to explain how you have solved the problem, rather than just posting a link.

Comment: This is great, thanks! What about if they want to know more details about the AI that I'm using? Is there a way that I can get them to ask more questions?

Answer: \begin{itemize}
\item I am an AI.
\end{itemize}AI: Answer: I'm an AI. How does it work? You can ask me about anything. Comment: Please also include your own code in the answer to explain how you have solved the problem, rather than just posting a link. Comment: This is great, thanks! What about if they 

### prompt template

In [1]:
# template = """
# You are an expert at providing concise and clear descriptions. 
# Describe the following topic in one sentence:

# Topic: {topic}
# """

# prompt = PromptTemplate.from_template(template)
# callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])


In [ ]:
llm = LlamaCpp(
    model_path=cfg.llm_gguf_model_path,
    n_gpu_layers=1, # Use m2 max chip, metal sets to 1.
    n_batch=512,  
    n_ctx=4096,     
    f16_kv=True,  
    callback_manager=callback_manager,
    # stop=["Question:"],
    verbose=False,
)

llm_chain = prompt | llm

ggml_metal_init: skipping kernel_get_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_1row              (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_l4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_bf16                  (not supported)
ggml_metal_init: skipping kernel_mul_mv_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_mul_mm_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mm_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h64           (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h80           (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h96           (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h112          (not supported)
ggml

In [ ]:
formatted_prompt = prompt.format(topic="dog")

# print(formatted_prompt)

response = llm.invoke(formatted_prompt)

print(response)


## 🌈 What is the subject of your description?

* **Topic**: dog (a pet)

## 🌈 What is the subject of your description?

* **Topic**: dog (a pet)



In [ ]:
response=llm_chain.invoke({"topic": 'cat'})


### Examples:  
*  "It's a feline."


In [ ]:
response=llm_chain.invoke({"topic": 'NBA'})


You have completed the topic, but before you submit it for review, you should proofread and check your spelling. 


In [ ]:
from langchain.schema.output_parser import StrOutputParser

llm_chain = prompt | llm | StrOutputParser()


response=llm_chain.invoke({"topic": 'NBA'})


### 0. What you are not sure about:

- You're not sure if this should be the "Topic" or a "Skill".
- You think it could possibly be a "Skill", but you want to give the topic some extra credit so we know it was given your attention, even though it is really more of a skill.


### try langchain_community.chat_models.ChatLlamaCpp

https://python.langchain.com/docs/integrations/chat/llamacpp/

In [ ]:
from langchain_community.chat_models import ChatLlamaCpp
from langchain_core.messages import HumanMessage


In [ ]:
llm = ChatLlamaCpp(
    temperature=0.5,
    model_path=cfg.llm_gguf_model_path_2,
    n_ctx=10000,
    n_gpu_layers=8,
    n_batch=300,  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    max_tokens=512,
    repeat_penalty=1.5,
    top_p=0.5,
    verbose=True,
)



llama_load_model_from_file: using device Metal (Apple M2 Max) - 21845 MiB free
llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from ../model_gguf/llama-2-7b-chat.Q4_0.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
l

In [ ]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]

ai_msg = llm.invoke(messages)
ai_msg




llama_perf_context_print:        load time =    3037.53 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    41 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    26 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    3997.73 ms /    67 tokens


AIMessage(content='  Sure! Here\'s the translation of "I love programming" in French:\nJe suis passionné par leprogramming.', additional_kwargs={}, response_metadata={'finish_reason': 'stop'}, id='run-a302fe2c-5515-42c4-aea7-cc96a3236252-0')

In [ ]:
print(ai_msg.content)

  Sure! Here's the translation of "I love programming" in French:
Je suis passionné par leprogramming.


In [ ]:
messages=[HumanMessage(content="Hi!")]
ai_msg = llm.invoke(messages)
ai_msg.content

Llama.generate: 1 prefix-match hit, remaining 12 prompt tokens to eval
llama_perf_context_print:        load time =    3037.53 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    12 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    26 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    1391.12 ms /    38 tokens


"  Hello! It's nice to meet you. hopefully, we can have a great conversation today! How are things going?"

In [ ]:
# chaining

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that translates {input_language} to {output_language}.",
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | llm
chain.invoke(
    {
        "input_language": "English",
        "output_language": "German",
        "input": "I love programming.",
    }
)

Llama.generate: 1 prefix-match hit, remaining 34 prompt tokens to eval
llama_perf_context_print:        load time =    3037.53 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    34 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    84 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    3728.62 ms /   118 tokens


AIMessage(content='  Great! I\'m happy to help you translate "I love programming" into German. Here is the translation:\n Ich liebe Programmiersachen (pronounced eek lee-bah prohr-mah-reez)\nSo, if someone asks how you feel about coding or computer science in Germany, this phrase can be used to express your enthusiasm and passion for programming.', additional_kwargs={}, response_metadata={'finish_reason': 'stop'}, id='run-07f02bd1-9d0d-474b-9ae9-b6267f3b9f37-0')

In [ ]:
from langchain_core.tools import tool
from pydantic import BaseModel, Field

In [ ]:

class WeatherInput(BaseModel):
    location: str = Field(description="The city and state, e.g. San Francisco, CA")
    unit: str = Field(enum=["celsius", "fahrenheit"])


@tool("get_current_weather", args_schema=WeatherInput)
def get_weather(location: str, unit: str):
    """Get the current weather in a given location"""
    return f"Now the weather in {location} is 22 {unit}"


llm_with_tools = llm.bind_tools(
    tools=[get_weather],
    tool_choice={"type": "function", "function": {"name": "get_current_weather"}}, # force invoke function
)



In [ ]:
ai_msg = llm_with_tools.invoke(
    "what is the weather like in HCMC in celsius",
)

Llama.generate: 1 prefix-match hit, remaining 23 prompt tokens to eval
llama_perf_context_print:        load time =    3037.53 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    23 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    22 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    1445.90 ms /    45 tokens


In [ ]:
ai_msg

AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_current_weather', 'arguments': '{"location": "Ho Chi Minh City, Vietnam", "unit":"Vietnam"}'}, 'tool_calls': [{'id': 'call__0_get_current_weather_cmpl-eab2f349-619f-4dc0-8cb8-2602a3d2109f', 'type': 'function', 'function': {'name': 'get_current_weather', 'arguments': '{"location": "Ho Chi Minh City, Vietnam", "unit":"Vietnam"}'}}]}, response_metadata={'token_usage': {'prompt_tokens': 24, 'completion_tokens': 22, 'total_tokens': 46}, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-0fffcdb0-5fd7-4acd-a500-1875e09da4ee-0', tool_calls=[{'name': 'get_current_weather', 'args': {'location': 'Ho Chi Minh City, Vietnam', 'unit': 'Vietnam'}, 'id': 'call__0_get_current_weather_cmpl-eab2f349-619f-4dc0-8cb8-2602a3d2109f', 'type': 'tool_call'}])

In [ ]:
ai_msg.content

''

In [ ]:
ai_msg.tool_calls

[{'name': 'get_current_weather',
  'args': {'location': 'Ho Chi Minh City, Vietnam', 'unit': 'Vietnam'},
  'id': 'call__0_get_current_weather_cmpl-eab2f349-619f-4dc0-8cb8-2602a3d2109f',
  'type': 'tool_call'}]

In [ ]:
class MagicFunctionInput(BaseModel):
    magic_function_input: int = Field(description="The input value for magic function")


@tool("get_magic_function", args_schema=MagicFunctionInput)
def magic_function(magic_function_input: int):
    """Get the value of magic function for an input."""
    return magic_function_input + 2


llm_with_tools = llm.bind_tools(
    tools=[magic_function],
    tool_choice={"type": "function", "function": {"name": "get_magic_function"}},
)

ai_msg = llm_with_tools.invoke(
    "What is magic function of 3?",
)

ai_msg

Llama.generate: 6 prefix-match hit, remaining 13 prompt tokens to eval
llama_perf_context_print:        load time =    3037.53 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    13 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    12 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    1004.87 ms /    25 tokens


AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_magic_function', 'arguments': '{ "magic_function_input":3}'}, 'tool_calls': [{'id': 'call__0_get_magic_function_cmpl-dd7986b7-a31d-4f4e-a57c-efc01344f0e5', 'type': 'function', 'function': {'name': 'get_magic_function', 'arguments': '{ "magic_function_input":3}'}}]}, response_metadata={'token_usage': {'prompt_tokens': 19, 'completion_tokens': 12, 'total_tokens': 31}, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-17611426-9d90-4825-a83f-b10cc33a5a21-0', tool_calls=[{'name': 'get_magic_function', 'args': {'magic_function_input': 3}, 'id': 'call__0_get_magic_function_cmpl-dd7986b7-a31d-4f4e-a57c-efc01344f0e5', 'type': 'tool_call'}])

In [ ]:
ai_msg.tool_calls

[{'name': 'get_magic_function',
  'args': {'magic_function_input': 3},
  'id': 'call__0_get_magic_function_cmpl-dd7986b7-a31d-4f4e-a57c-efc01344f0e5',
  'type': 'tool_call'}]

In [ ]:
from langchain_core.utils.function_calling import convert_to_openai_tool
from pydantic import BaseModel


class Joke(BaseModel):
    """A setup to a joke and the punchline."""

    setup: str
    punchline: str


dict_schema = convert_to_openai_tool(Joke)
structured_llm = llm.with_structured_output(dict_schema)
result = structured_llm.invoke("Tell me a joke about birds")
result

Llama.generate: 6 prefix-match hit, remaining 13 prompt tokens to eval
llama_perf_context_print:        load time =    3037.53 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    13 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    76 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    3694.44 ms /    89 tokens


{'setup': "Why did the bird go to therapy? It had a fowl mood! 😂}\n Unterscheidung: Bird jokes can be egg-cellent, but they're not always in flight. This one took off with me though!",
 'punchline': './laughter'}

In [ ]:
result

{'setup': "Why did the bird go to therapy? It had a fowl mood! 😂}\n Unterscheidung: Bird jokes can be egg-cellent, but they're not always in flight. This one took off with me though!",
 'punchline': './laughter'}

In [ ]:
for chunk in llm.stream("what is 25x5"):
    print(chunk.content, end="\n", flush=True)

Llama.generate: 6 prefix-match hit, remaining 12 prompt tokens to eval



 
 Sure
!
 The
 answer
 to
 `
2
5
x
5
`
 is
:


 everybody
 lov
es
 math
.


llama_perf_context_print:        load time =    3037.53 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    12 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    20 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    1625.40 ms /    32 tokens


#### could find that the llm has invoked the tool. however, it does not return any content. It may be about the langchain agent, so i need to write code snippet to make the llm return the content with info from tool.

https://python.langchain.com/docs/tutorials/agents/

In [ ]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import (AgentExecutor,create_react_agent,ZeroShotAgent)

from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.messages import SystemMessage

In [ ]:
# 定义工具列表
tools = [
    Tool(
        name="get_current_weather",
        func=get_weather,
        description="Get the current weather in a given location"
    )
]

In [ ]:
template_messages = [
    SystemMessage(content="You are a helpful assistant."),
    # MessagesPlaceholder(variable_name="chat_history"),
    # MessagesPlaceholder(variable_name='agent_scratchpad'),
    HumanMessagePromptTemplate.from_template("{text}"),
    # MessagesPlaceholder(variable_name="tools"),            
    # MessagesPlaceholder(variable_name="tool_names"), 
]
prompt_template = ChatPromptTemplate.from_messages(template_messages)

In [ ]:
agent=initialize_agent(tools=tools,llm=llm,agent="conversational-react-description",verbose=True,prompt=prompt_template)

In [ ]:
# get weather through agent
response = agent.run({"input":"what is the weather like in HCMC in celsius","chat_history": []})
print(response)



> Entering new AgentExecutor chain...


Llama.generate: 6 prefix-match hit, remaining 453 prompt tokens to eval
llama_perf_context_print:        load time =    2231.93 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   453 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    49 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   10828.38 ms /   502 tokens


  Thought: Do I need to use a tool? No
AI: The current temperature in Ho Chi Minh City is around 28 degrees Celsius. Would you like me to provide more information or assist with something else?

> Finished chain.
The current temperature in Ho Chi Minh City is around 28 degrees Celsius. Would you like me to provide more information or assist with something else?


#### Here shows a problem that i want the agent to invoke the tool but it did not. I need to fix the problem so that the agent could know it needs to invoke corresponding tools sometimes.

In [ ]:
# create Agent
agent = create_react_agent(llm=llm, tools=tools,prompt=prompt_template)

In [ ]:
# create AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [2]:
# invoke agent_executor to automatically handle tool calls

response = agent_executor.invoke({"text": "What is the weather like in HCMC in celsius?"})


print(response)

In [ ]:
class MagicFunctionInput(BaseModel):
    magic_function_input: int = Field(description="The input value for magic function")


@tool("get_magic_function", args_schema=MagicFunctionInput)
def magic_function(magic_function_input: int):
    """Get the value of magic function for an input."""
    return magic_function_input + 2


llm_with_tools = llm.bind_tools(
    tools=[magic_function],
    tool_choice={"type": "function", "function": {"name": "get_magic_function"}},
)

ai_msg = llm_with_tools.invoke(
    "What is magic function of 3?",
)

ai_msg

Llama.generate: 6 prefix-match hit, remaining 13 prompt tokens to eval
llama_perf_context_print:        load time =    4123.12 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    13 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    12 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    3170.33 ms /    25 tokens


AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_magic_function', 'arguments': '{ "magic_function_input":3}'}, 'tool_calls': [{'id': 'call__0_get_magic_function_cmpl-ed4bc443-9a17-4f70-8ad4-6a699e5a767d', 'type': 'function', 'function': {'name': 'get_magic_function', 'arguments': '{ "magic_function_input":3}'}}]}, response_metadata={'token_usage': {'prompt_tokens': 19, 'completion_tokens': 12, 'total_tokens': 31}, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-7f973e88-8e37-4dca-9b5f-1fd51a32f869-0', tool_calls=[{'name': 'get_magic_function', 'args': {'magic_function_input': 3}, 'id': 'call__0_get_magic_function_cmpl-ed4bc443-9a17-4f70-8ad4-6a699e5a767d', 'type': 'tool_call'}])

In [ ]:
ai_msg.tool_calls

[{'name': 'get_magic_function',
  'args': {'magic_function_input': 3},
  'id': 'call__0_get_magic_function_cmpl-ed4bc443-9a17-4f70-8ad4-6a699e5a767d',
  'type': 'tool_call'}]

In [ ]:
from langchain_core.utils.function_calling import convert_to_openai_tool
from pydantic import BaseModel


class Joke(BaseModel):
    """A setup to a joke and the punchline."""

    setup: str
    punchline: str


dict_schema = convert_to_openai_tool(Joke)
structured_llm = llm.with_structured_output(dict_schema)
result = structured_llm.invoke("Tell me a joke about birds")
result

Llama.generate: 6 prefix-match hit, remaining 13 prompt tokens to eval
llama_perf_context_print:        load time =    4123.12 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    13 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    36 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    4493.47 ms /    49 tokens


{'setup': 'Why did the bird go to therapy?',
 'punchline': 'Because it had a fowl mood!'}

In [ ]:
result

{'setup': 'Why did the bird go to therapy?',
 'punchline': 'Because it had a fowl mood!'}

In [ ]:
for chunk in llm.stream("what is 25x5"):
    print(chunk.content, end="\n", flush=True)

Llama.generate: 17 prefix-match hit, remaining 1 prompt tokens to eval


 Sure
!
 The
 answer
 to
 `
2
5
x
5
`
 is
:


 everybody
 lov
es
 math
.


llama_perf_context_print:        load time =    4123.12 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    21 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    1107.35 ms /    22 tokens
